In [38]:
#!pip install openai
from openai import OpenAI
import pandas as pd
import os
import numpy as np
import math
from unidecode import unidecode
import re

def get_subdirectories(path):
    subdirectories = [path+'/'+name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
    return subdirectories
    
def get_md_file(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".md"):
            return filename
    return None

def get_l_art(path = "C:/Users/1/!hackatone-c1/papers/MD_pdf"):
    sd = get_subdirectories(path)
    l_art=[]
    for i in range(len(sd)):
         md = get_md_file(sd[i])
         l_art.append(sd[i]+'/'+md)
    return l_art   

def check_genes(articles, genes):
    results = np.full((len(articles), len(genes)), False, dtype='bool')
    for a in range(len(articles)):
        file_data = open(articles[a], 'rb').read()
        for g in range(len(genes)):
            #print(genes[g])
            match = re.findall(genes[g], str(file_data))  
            if match!=[]:
                #print(a,g,match)
                results[a,g] = True
    return results

def get_unique(data_names):
    unique = []
    for a in data_names:
        if not a in unique:
            unique.append(a)
    return unique
    
def prepare_list_genes(df):
    lg = df['hgnc'].tolist()
    ug = get_unique(lg)
    return ug

def filter_list(l):
    ll=[]
    for i in range(len(l)):
        if isinstance(l[i], str):
             ll.append(l[i])
    return ll        

#ген-набор статей где он встречается
def make_list_articles(genes, articles, r):
    #print(len(genes))
    lg=[]
    la=[]
    for i in range(len(genes)):
        laa=[]
        #print(np.sum(r[:,i]))
        if np.sum(r[:,i])>1:
            #print(genes[i])
            lg.append(genes[i])
            for j in range(len(articles)):
                if r[j,i]==True:
                    #print(articles[j])
                    laa.append(articles[j])
            la.append(laa)         
    return lg, la

def check_importance(gene, l, thr=4):
    req = prepare_request_nart(gene, l)
    resp = send_request(req, temperature=0)
    print("resp:",resp)
    v = convert_to_number(resp) 
    if v>=thr:
        return True
    else:
        return False
    
#гены-набор статей где они встречаются
def make_list_genes_articles(genes, articles, r):
    #ma = np.sum(r[:,0]&r[:,1]);ii=0;jj=1
    lg_pair = [];la_pair = []
    for i in range(len(genes)):
        for j in range(i+1,len(genes)):
            ma=np.sum(r[:,j]&r[:,i])
            if ma>1:
                print("ma:",ma)
                la =[]
                for a in range(len(articles)):
                    if r[a, i]==True:
                        la.append(articles[a])   
                f1 = check_importance(genes[i], la, thr=4)
                f2 = check_importance(genes[j], la, thr=4)
                if f1 and f2:    
                    lg_pair.append([genes[ii],genes[jj]])
                    la_pair.append(la)      
    return lg, la


def prepare_request_nart(g, la, long=False):
    s = "Ген " + g + " изучался в " + str(len(la)) + " статьях:\n"
    for a in range(len(la)):
        s = s + "Статья " + str(a+1) + ":"
        #articles = la[a]
        file_data = open(la[a], 'rb').read()
        s=s+ str(file_data) + "\n"  
    s = s + " На основе этих публикаций предскажи, к какому уровню уверенности"
    s = s + "(Highest,High,Моderate,Low,Lowest) должен быть отнесён этот ген в базе OpenGenes? "  
    if long==False:
        s = s + "В ответе приведи только одну оценку только для этого гена."     
    return s

def prepare_request(g, art, long=False):
    s = "Ген " + g + " изучался в данной статье:\n"
    file_data = open(art, 'rb').read()
    s=s + str(file_data) + "\n"  
    s = s + " На основе этой публикации предскажи, к какому уровню уверенности"
    s = s + "(Highest,High,Моderate,Low,Lowest) должен быть отнесён этот ген в базе OpenGenes? "  
    if long==False:
        s = s + "В ответе приведи только одну оценку только для этого гена."     
    return s

def prepare_request_cross(g, la, long=False):
    s = "Ряд генов изучался в нескольких статьях:\n"
    for a in range(len(la)):
        s = s + "Статья " + str(a+1) + ":"
        file_data = open(la[a], 'rb').read()
        s=s+ str(file_data) + "\n"  
    #s = s + " Какие гены были важны в данных исследованиях? "
    s = s + " Какие гены были общие в данных исследованиях? Напиши только два гена"
    #if long==False:
    #    s = s + "В ответе приведи только один ген"     
    return s


def convert_to_number(level, default=0):
    levels = {
        "Highest": 5,
        "High": 4,
        "Moderate": 3,
        "Low": 2,
        "Lowest": 1
    }
    return levels.get(unidecode(level), default)


def send_request(req, temperature=0.5,max_tokens=75):
    client = OpenAI(
        base_url="http://80.209.242.40:8000/v1",
        api_key="dummy-key"
    )
    response = client.chat.completions.create(
        model="llama-3.3-70b-instruct",
        temperature=temperature,
        max_tokens=max_tokens,   #2048
        messages=[
            {"role": "user", "content": req}
        ]
    )    
    return response.choices[0].message.content

# отклонение средней оценки каждой статьи от оценки всех статей
# суммарно по всем генам
def calc_metric(articles, genes):
    gm = check_genes(articles, genes)
    lg, la = make_list_articles(genes, articles, gm)
    print('Генов в нескольких статьях:', len(lg))
    print(lg)
    gmetrics=[]
    for i in range(len(lg)):
        ratings=[]
        #1 ген-1статья
        for j in range(len(la[i])):
            l = la[i]
            #print(l)
            req = prepare_request(lg[i], l[j])
            resp = send_request(req, temperature=0)
            v = convert_to_number(resp)
            #print(v)
            ratings.append(v)

        #1 ген- все статьи
        #print(lg[i])
        req = prepare_request_nart(lg[i],la[i])
        resp = send_request(req, temperature=0)
        v = convert_to_number(resp)
        #print(v)
        gmetric = abs(v-np.mean(ratings)) 
        gmetrics.append(gmetric)

    #print(gmetrics)
    metric = np.sum(gmetrics)
    #print(metric)
    return metric

# наличие в результате ссылок на гены упомянутые в других статьях
def calc_metric_crossgenes(articles, genes):
    gm = check_genes(articles, genes)
    lg,la = make_list_genes_articles(genes, articles, gm)
    
    #узнали важность генов
    print(lg[0])
    req = prepare_request_nart(lg[0],la)
    resp = send_request(req, temperature=0)
    v = convert_to_number(resp)
    print(v)
    print(lg[1])
    req = prepare_request_nart(lg[1],la)
    resp = send_request(req, temperature=0)
    v = convert_to_number(resp)
    print(v)
 
    #проверили что в выдаче есть общие гены
    s=0
    gmetrics=[]
    #1 ген- все статьи
    print(lg)
    req = prepare_request_cross(lg[0], la, long=False)
    resp = send_request(req,max_tokens=75)
    v = unidecode(resp)
    print(v)
    if lg[1] in v:
        s=s+.5
    req = prepare_request_cross(lg[1], la, long=False)
    resp = send_request(req,max_tokens=75)
    v = unidecode(resp)
    print(v)
    if lg[0] in v:
        s=s+.5
        
    gmetrics.append(s)
    metric = np.mean(gmetrics)
    #print(metric)
    return s



In [29]:

#aging_mechanisms = pd.read_csv("open_genes/gene-aging-mechanisms.tsv",sep=r'[,\t]')
aging_mechanisms = pd.read_csv("open_genes/gene-aging-mechanisms.tsv",sep='\t')
#aging_mechanisms = aging_mechanisms.dropna()

diseases = pd.read_csv("open_genes/gene-diseases.tsv",delimiter='\t')
criteria = pd.read_csv("open_genes/gene-criteria.tsv",delimiter='\t')

confidence_level = pd.read_csv("open_genes/gene-confidence-level.tsv",delimiter='\t')
regulation = pd.read_csv("open_genes/gene-regulation.tsv",delimiter='\t')
associations_with_longevity = pd.read_csv("open_genes/associations-with-longevity.tsv",delimiter='\t')
age_related_processes_change = pd.read_csv("open_genes/age-related-processes-change.tsv",delimiter='\t')
age_related_changes = pd.read_csv("open_genes/age-related-changes.tsv",delimiter='\t')
confidence_level = pd.read_csv("open_genes/lifespan-change.tsv",delimiter='\t')

articles = get_l_art()
print("Cтатей:",len(articles))
genes = prepare_list_genes(associations_with_longevity)

print("Associated with longevity:")
print("Генов:",len(genes))
#r = check_genes(articles, genes)
#genes = prepare_list_genes(age_related_processes_change)
#print(len(genes))
#r = check_genes(articles, genes)

print(calc_metric(articles, genes))

genes = aging_mechanisms.index.tolist()
#print(len(genes))
genes =filter_list(genes)
print("Gene aging mechanisms:")
print("Генов:",len(genes))

#print(calc_metric(articles, genes))
print(calc_metric_crossgenes(articles, genes))


Cтатей: 6
Associated with longevity:
Генов: 367
Генов в нескольких статьях: 1
['GHR']
0.0
Gene aging mechanisms:
Генов: 2404
статей совпало: 2
0 1584
GHR
3
CS
3
['GHR', 'CS']
Sirt6 i PAPP-A
Sirt6 i PAPP-A
0


In [39]:

gm = check_genes(articles, genes)
lg,la = make_list_genes_articles(genes, articles, gm)

print(lg, la)
      
    

ma: 2
resp: Моderate
resp: Moderate
ma: 2
resp: Lowest
resp: Lowest
['GHR', 'CS'] ['C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1186_s12979-015-0043-y/10.1186_s12979-015-0043-y.md', 'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1371_journal.pone.0120048/10.1371_journal.pone.0120048.md', 'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.7554_eLife.45974/10.7554_eLife.45974.md']


In [42]:
## print(np.argmax(lg))
print(lg[12])
print(la[12])


['GHR', 'GHRH', 'IGF1', 'SIRT1', 'HDAC2', 'SIRT6', 'C2', 'CS', 'SRI']
['C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1073_pnas.1016306107/10.1073_pnas.1016306107.md', 'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1111_acel.12624/10.1111_acel.12624.md', 'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1371_journal.pone.0120048/10.1371_journal.pone.0120048.md', 'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.7554_eLife.45974/10.7554_eLife.45974.md']


In [26]:
lm

[['C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1073_pnas.1016306107/10.1073_pnas.1016306107.md',
  'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1111_acel.12624/10.1111_acel.12624.md',
  'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1126_science.1251141/10.1126_science.1251141.md',
  'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1186_s12979-015-0043-y/10.1186_s12979-015-0043-y.md',
  'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1371_journal.pone.0120048/10.1371_journal.pone.0120048.md',
  'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.7554_eLife.45974/10.7554_eLife.45974.md'],
 ['C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1073_pnas.1016306107/10.1073_pnas.1016306107.md',
  'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1111_acel.12624/10.1111_acel.12624.md',
  'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1126_science.1251141/10.1126_science.1251141.md',
  'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1186_s12979-015-0043-y/10.1186_s12979-015-0043-y.md',
  'C:/Users/1/!hackatone-c1/papers/MD_pdf/10.1371_journal.p